In [ ]:
!pip install -U tf2onnx onnx

import os
import tensorflow as tf
from tensorflow import keras
import tf2onnx


In [ ]:
import os
print(os.listdir("/kaggle/input/my-xray-modell/"))


In [ ]:
import tensorflow as tf
from tensorflow import keras
import tf2onnx

# Load Keras model
model_path = "/kaggle/input/my-xray-modell/xception_pneumonia_model.keras"
model = keras.models.load_model(model_path)

# Define input signature
spec = (tf.TensorSpec((None, 150, 150, 3), tf.float32, name="input"),)

# ONNX save path
onnx_path = "/kaggle/working/xception_pneumonia.onnx"

# Convert to ONNX
tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13,
    output_path=onnx_path
)

print("✅ Conversion complete!")


In [ ]:
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))


In [ ]:
import onnx

onnx_model_path = "/kaggle/working/xception_pneumonia.onnx"
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")



In [ ]:
!pip install -q onnxruntime


In [ ]:
import onnxruntime as ort
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import preprocess_input

# Load ONNX model
session = ort.InferenceSession("/kaggle/working/xception_pneumonia.onnx")

# Load image (FIX THIS PATH)
img_path = "/kaggle/input/patient-xray-test/journal.pone.0256630.g001.PNG"
img = load_img(img_path, target_size=(150,150))

# Preprocess
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x).astype(np.float32)

# Predict
input_name = session.get_inputs()[0].name
output = session.run(None, {input_name: x})

label = "PNEUMONIA" if output[0][0][0] > 0.5 else "NORMAL"
print("Prediction:", label)

# Show image
plt.imshow(img)
plt.title(f"Predicted: {label}")
plt.axis("off")
plt.show()


In [ ]:
import os
os.listdir("/kaggle/working")
